# Homework 3

In [1]:
import numpy as np

---

## 6.

In [43]:
data = np.array([(0, 3), (1, 5), (2, -1), (3, 2)], dtype=np.float32)

def p(degree, x):
    n = degree + 1
    xd, yd = data[:n].T
    c = yd.copy()
    for i in range(1, n + 1):
        for j in range(n - 1, i - 1, -1):
            c[j] = (c[j] - c[j-1])/(xd[j] - xd[j-i])
    r = c[0]
    for j in range(1, n):
        r += c[j]*np.prod(x - xd[:j])
    return r

In [44]:
for degree in (1, 2, 3):
    r = p(degree, 0.05)
    print('Degree %i: p(%.2f) = %.6f' % (degree, 0.05, r))

Degree 1: p(0.05) = 3.100000
Degree 2: p(0.05) = 3.290000
Degree 3: p(0.05) = 3.552438


----

## 7.

$$
\int_{0.5}^{1}x^4 dx = \frac{x^5}{5}\big|_{0.5}^1 = \frac{1 - 0.5^{5}}{5} \approx 0.19375
$$

In [54]:
def trapz(f, a, b):
    return 0.5*(f(b) + f(a))*(b - a)

In [55]:
exact = 0.19375
a = 0.5
b = 1.0
f = lambda x: x**4
r = trapz(f, a, b)
relerr = 100*abs(r - exact)/exact
print('Result: %.5f' % r)
print('Relative Error: %.5f%%' % relerr)

Result: 0.26562
Relative Error: 37.09677%


## 8.

$$
\int_1^2 x \ln x ~dx \approx 0.636294
$$

In [62]:
exact = 0.636294
n = 3
a = 1.0
b = 2.0
h = (b - a)/n
points = [i*h + a for i in range(n + 1)]
points

[1.0, 1.3333333333333333, 1.6666666666666665, 2.0]

In [64]:
f = lambda x: np.log(x)*x
r = 0
for i in range(n):
    r += trapz(f, points[i], points[i + 1])

In [65]:
relerr = 100*abs(r - exact)/exact
print('Result: %.5f' % r)
print('Relative Error: %.5f%%' % relerr)

Result: 0.64270
Relative Error: 1.00673%


## 9.

In [66]:
A = np.array([
    [-4, 14, 0],
    [-5, 13, 0],
    [-1, 0, 0],
])
x0 = np.ones(3)
exact = np.array([6, 3, 1])